In [1]:
import pyspark

sc = pyspark.sql.SparkSession.Builder().getOrCreate()

## Levanto los archivos

In [2]:
import json

In [5]:
points_recep_df = sc.read.json('datos/points-recep.jsonlines')

### Agrego datos de la 4ta antena a todos las emisiones.

In [31]:
join=points_recep_df.drop('recep_3').join(points_recep_df.filter('size(recep_3) > 0').select('Punto', 'recep_3'), 'Punto', 'left_outer')




In [35]:
def fix_null_recep(row):
    row_dict = row.asDict()
    if row_dict['recep_3'] is None:
        row_dict['recep_3'] = []
    return row_dict

points_recep = join.rdd.map(fix_null_recep)

In [36]:
points_recep.take(1)

[{'Fecha': u'27/01/2018',
  'Fin': u'17:57:00',
  'Inicio': u'17:55:00',
  'Punto': 7,
  'Tag': u'12',
  'recep_0': [],
  'recep_1': [],
  'recep_2': [],
  'recep_3': [],
  'timestamp_fin': u'2018-01-27 17:57:00',
  'timestamp_inicio': u'2018-01-27 17:55:00',
  'x': 463183.096026691,
  'y': 6111738.93734562}]

In [37]:
non_empty_points_recp = points_recep.filter(lambda x: len(x['recep_0']+x['recep_1']+x['recep_2']+x['recep_3']) > 0)


### Me quedo con una sola emision por punto

In [50]:
unique_points = non_empty_points_recp.groupBy(lambda x: x['Punto']).map(lambda x: list(x[1])[0])

### Regresion por eje x

In [100]:
import numpy as np

def generate_attrs(row):
    #f = lambda l: len(l)
    f = lambda l: float(np.mean(l) if len(l) > 0 else 0)
    data = {
        'recep_0': f(row['recep_0']),
        'recep_1': f(row['recep_1']),
        'recep_2': f(row['recep_2']),
        'recep_3': f(row['recep_3'])
    }
    target = row['y']
    return {'data': data, 'target': target}
    
points = unique_points.map(generate_attrs)

In [102]:
import pandas as pd

points_target = pd.DataFrame(points.map(lambda x: x['target']).collect())
points_data = pd.DataFrame(points.map(lambda x: x['data']).collect())

In [104]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

clf = KNeighborsRegressor(n_neighbors=5)
scores = cross_val_score(clf, points_data, points_target, cv=5, scoring='neg_mean_absolute_error')
print('cross val mae: {}'.format(scores))

cross val mae: [-461.36813832 -377.50476812 -333.01003056 -194.32327923 -214.0366668 ]


### Clasificacion de puntos

In [63]:
def generate_attrs(row):
    #f = lambda l: len(l)
    f = lambda l: float(np.mean(l) if len(l) > 0 else 0)
    data = {
        'recep_0': f(row['recep_0']),
        'recep_1': f(row['recep_1']),
        'recep_2': f(row['recep_2']),
        'recep_3': f(row['recep_3'])
    }
    target = row['x']
    return {'data': data, 'target': row['Punto']}
    
points = unique_points.map(generate_attrs)


import pandas as pd

points_target = pd.DataFrame(points.map(lambda x: x['target']).collect())
points_data = pd.DataFrame(points.map(lambda x: x['data']).collect())



In [99]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold 
import math
dict_coordenadas = points_recep.map(lambda x: (x['Punto'],(x['x'], x['y']))).collectAsMap()
global dict_coordenadas

def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)

def calculate_mae_distance(predictions, real):
    sum_error = 0
    for i in range(len(predictions)):
        pred_position = dict_coordenadas[predictions[i]]
        real_position = dict_coordenadas[real[i]]
        sum_error += distance(pred_position, real_position)
    return sum_error/len(predictions)
    

points_data_np = np.array(points_data)
points_target_np = np.array(points_target).ravel()
    


clf = KNeighborsClassifier(n_neighbors=5)
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(points_data):
    X_train, X_test = points_data_np[train_index], points_data_np[test_index]
    y_train, y_test = points_target_np[train_index], points_target_np[test_index]
    
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    mae = calculate_mae_distance(predictions, y_test)
    print('mean absolute error {}'.format(mae))



mean absolute error 456.709923989
mean absolute error 574.024182072
mean absolute error 545.169045405
mean absolute error 525.940451002
mean absolute error 469.286738142
